<a href="https://colab.research.google.com/github/Lucy-Moctezuma/SFSU-CodeLab-Work-/blob/main/E.%20Coli%20Machine%20Learning%20Project/1_Loading_and_Preparing_Data_for_ML_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Welcome to Antibiotic Resistance Prediction for E.Coli using Machine Learning**

### ***Summary***:

Antibiotic resistance is a global public health concern. Bacteria are evolving resistance to the current prescribed antibiotics resulting in strains developing multi-drug resistance. Currently, clinics often perform traditional culture-based assays (i.e., testing whether a drug would kill the bacteria in a petri dish) to determine antibiotic resistance in bacterial strains. Alternatively, clinics can also sequence these strains. These sequences can be analyzed to predict resistance to a prescribed antibiotic. There are different ways to perform the analysis and Machine Learning is one of them. This series of tutorials intends to help you understand how to do such an analysis.

We will process publically available whole genome sequences of *E. coli* strains to create Logistic Regression, Random Forest, Gradient Boosted Trees, and Neural Network models to predict **Resistance (R)** and **Susceptibility (S)** for each strain. The strains have already been tested in the lab, so we will
later be able to compare the predictions made by our Machine Learning models with the traditional culture-based assays results in order to determine the performance for each of these models.

The data and the approach we take are from a previously published paper (Moradigaravand 2018).

<a name="cell-id"></a>
### ***Data origin***:
**Publication**:
Moradigaravand D, Palm M, Farewell A, Mustonen V, Warringer J, Parts L (2018) Prediction of antibiotic resistance in *Escherichia coli* from large-scale pan-genome data. PLoS Comput Biol 14(12): e1006258. https://doi.org/10.1371/journal.pcbi.1006258

**Github page**: https://github.com/DaneshMoradigaravand/PanPred

### ***Objectives of this Notebook***:

- The Objective of this first notebook is to explore and undertand the data we will be using as features to predict susceptibility (S) or resistance (R) in *E.coli* Bacteria for several drugs.
This tutorial will use the features of **Year of Isolation (Y)**, **Gene Absence or Presence (G)** and **Population Structure (S)**.

- In addition we will see in this notebook how to clean up and pre-process our data before feeding it into different machine learning algorithms.

## **1) Importing all necessary packages for Dataframe Manipulation**

The code below will allow you to import the packages needed to load and pre-process the data used for our models.

**NOTE:** Please allow access to your google drive when prompted, this will let you create and store the files in your drive to be accessed later by subsequent notebooks as we make progress towards getting our final results.

In [ ]:
# Data Wrangling Imports
import pandas as pd
import numpy as np
from functools import reduce

# File Manipulation Imports
import os
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


## **2) Loading all Datasets used for Predictions**

The code below will load the datasets we use from Moragadivand's github page (link located in the [Data origin](#cell-id) section). There are three datasets used:

**a) Metadata**: Year of isolation and results from Antimicrobial Susceptibility Testing.

**b) Gene Presence and Absence**: Not all *E. coli* strains carry the same genes. We have a list of genes and information for each strain on whether it carries that gene.

**c) Population Structure**: Clusters based on SNP distance (number of differing sites) based on the core genome.

In [ ]:
# assign to url variable for each csv file
metadata_url = 'https://raw.githubusercontent.com/DaneshMoradigaravand/PanPred/master/test_data/Metadata.csv'
gene_presence_url = 'https://raw.githubusercontent.com/DaneshMoradigaravand/PanPred/master/test_data/AccessoryGene.csv'
pop_struc_url = 'https://raw.githubusercontent.com/DaneshMoradigaravand/PanPred/master/test_data/PopulationStructure.csv_labelencoded.csv'


# load the three csv files into the notebook
metadata = pd.read_csv(metadata_url)
gene_presence_data = pd.read_csv(gene_presence_url)
pop_struc_data = pd.read_csv(pop_struc_url)

### **a) Metadata:**
**Columns Summary:**

- **Isolate number**: Unique number assigned to identify a particular strain of *E. coli* Bacteria. Thus we will refer to each row of our dataset as an "Isolate" from now on.
- **Year of Isolation**: The year in which a particular bacterial isolate was collected from a patient.
- **Antibiotic drug**: There are 12 antibiotic drug columns named after their 3 letter abbreviation adopted from the "British Society of Antimicrobial Chemotherapy".

|Abreviation|Class: Subclass|Full name|
|:----------|:--------------|:--------|
|**CTZ**|Beta-lactams: Cephalosporins|Ceftazidime|
|**CTX**|Beta-lactams: Cephalosporins|Cefotaxime |
|**CXM**|Beta-lactams: Cephalosporins|Cefuroxime|
|**CET**|Beta-lactams: Cephalosporins|Cephalothin|
|**AMP**|Beta-lactams: Penicillin|Ampicillin|
|**AMX**|Beta-lactams: Penicillin|Amoxicillin|
|**AMC**|Beta-lactams: Penicillin|Amoxicillin + Clavulanate potassium|
|**TZP**|Beta-lactams: Piperacillin|Tazobactam|
|**GEN**|Aminoglycosides|Gentamicin|
|**TBM**|Aminoglycosides|Tobramycin|
|**TMP**|Antifolate|Trimethoprim|
|**CIP**|Fluoroquinolones|Ciprofloxacin|


In [ ]:
# Visualize the first 5 rows of our dataframe
metadata.head()


**Note**: NaN was used to mark when there is no data for that isolate and drug.

**Determination of Resistant(R) and Susceptible(S) labels:**

- **Antimicrobial Susceptibility Testing**:
Laboratory test where a bacterial isolate is grown in the presence of different concentrations of a drug to determine whether it is Susceptible(S), Intermediate (I) or Resistance(R).

- **Clinical Breakpoints**: Each of the drugs listed in the chart have a different concentration (clinical breakpoint) used to determine the Susceptibility or Resistance of *E. coli* to that drug. If the bacterium can grow at a drug concentration higher than the breakpoint, it is considered resistant.

- Results from the laboratory tests were determined based on the guidelines from the [European Committee on Antimicrobial Susceptibility Testing (EUCAST)](https://www.eucast.org/videos_and_online_seminars/english) on 25/01/2017. Click on the link to see a series of videos on how the laboratory tests are done.

- For this study, isolates that were classified as Intermediate(I) were lumped together with the Resistant ones.

In [ ]:
# Observe the shape and size of the dataframe
metadata.shape

The code above can show us the shape of our entire dataframe in the following format: **(Row count, Column count)**

**a) Row count**: There is a total of 1936 Isolates of *E. coli* Bacteria.

**b) Column count**: is 14 = Isolate number (1) + year (1)+ antibiotic laboratory test results (12)


### **b) Gene presence and absence:**

**Genes:** Sequences of DNA that (usually) code for proteins. These sequences can vary in length and the resulting proteins have a variety of functions. Some genes and their resulting proteins are well characterized and have names, whereas others are not well characterized. For our analysis, the genes from our data set will be generally classified into:

- **Core Genome:** These are the genes that are present in almost all individuals for a particular species. In our case, they are the genes that all our *E.coli* isolates have in common.

- **Accessory Genes:** These are the genes that might be found in one individual but may not be found in another, within the same species. These would be the genes that are unique to each of our *E. coli* isolate.

- **Pan-genome:** These are all the possible genes that can be found in a particular species. That is they are all the gene presents present in our *E. coli* isolates (Pan-genome = Core Genome + Accesory Genes).

The code below will show us the dataframe containing the presence and absence of all the genes detected in for each isolate.
- **0** = Absence of the gene
- **1** = Presence of the gene.

In [ ]:
# Visualize the first 5 rows of dataframe
gene_presence_data.head()

As mentioned above, not all genes are well known and have names, therefore the author has separated them in 2 sets of genes that were named differently:

**1) Known genes**: which were extracted from the annotated DNA sequences.

In the code below we can take a look at all the names of the named genes because **they do not have the word "group"** in them.

In [ ]:
# this code will output the list of named genes and the lenght of it
coding_genes = [col for col in gene_presence_data.drop(columns=["Unnamed: 0"]).columns if 'group' not in col]
print("List of coding_genes:")
print(coding_genes)
print("Total number of named genes included: ", len(coding_genes))

**2) Unknown genes**: these the DNA sequences that look like a gene (e.g., they have a start and a stop codon) and are grouped based on **orthologous gene groups**. This means that these are sequences where we don't necessarily know their function, but we know that they exist in many of the *E. coli* isolates, **they get a name that starts with "group"**.

**Orthologous genes**: are genes that derive from the same ancestral gene.





In [ ]:
# this code will output all the genes without an assigned gene name.
other_genes = [col for col in gene_presence_data.drop(columns=["Unnamed: 0"]).columns if 'group' in col]
print("List of unnamed genes:")
print(other_genes)
print("Total number of unnamed genes included: ", len(other_genes))

**NOTE:** The code below for instance is showing us which Isolates have the ortholog gene group called **"group_13605"** In this case we have 8 isolates that have this gene.

In [ ]:
# the code below prints all the isolate numbers that have a 1 (presence) for a particular group
print(gene_presence_data[gene_presence_data["group_13605" ]== 1]["Unnamed: 0"])

The code below can show us the shape of our entire dataframe in the following format: **(Row count, Column count)**

**a) Row count**: Notice that the number of isolates is 2033, whereas in the Metadata file there were only 1936. We can only work with isolates fo which we have metadata, so later on we will filter out the ones that do not have a corresponding metadata.

**b) Column count**: The number of columns is 17199 = Isolate number(1) + coding_genes(3815) + other_genes(13383)

In [ ]:
# Observe the shape and size of the dataframe
gene_presence_data.shape

**NOTE:** There are more observations in the gene presence and absence dataset (2033), we will later match these observations with the ones present in the metadata (1936) by Isolate number, and thus eliminate the ones that do not have a match.

### **c) Population structure data:**

**Population structure** is defined as the organization of genetic variation in a population or species. It is basically about how similar (genetically) different isolates are. The idea is that isolates that are genetically similar, are maybe likely to have the same resistance profile.

In [ ]:
# Visualize the first 5 rows of dataframe
pop_struc_data.head()

**1) Unnamed:0** column is the same as the **Isolate** column from the Metadata csv file.

**2) cutoff_#**: the names of this set of columns represent the different cutoff values used to group isolates into different clusters. For grouping the isolates in clusters the core genome sequences were compared between all pairs of isolates.
For instance cutoff value of 3 means that we have grouped two isolates together in a cluster if they have 3 or fewer differences (aka **SNPs**) between the two isolates. At the end of the process each isolate is classified into a different cluster depending on the cutoff value. Thus, each cutoff value should produce a different set of clusters.

**The numbers below each column are simply the reference number for each cluster, these are arbitraty.** For Example: for the Isolate number `11657_5#11` for the `cutoff_2` (2 or less SNPs) belongs to the cluster number 1045, but for the  `cutoff_3` (3 or less SNPs) belongs to the cluster number 1043.

**NOTE:** **SNP (Single Nucleotide Polymorphism)** is a single difference on a one nucleotide in a particular gene when we have a pair of isolates being compared.

![SNP Picture.jpg](https://drive.google.com/uc?export=view&id=185p-fkkbtxiwu4TkZa6AZ6Dr6yya7pp6)




If the cutoff value is low, we can expect that most isolates will not be grouped with any other isolate, making a higher number of clusters but each cluster would be composed of a small number of isolates. The sum of all the members for each cluster should yield our total number of isolates (i.e, 1936).
- For example for the **cutoff_2008** we are considering all isolates that have 2008 or less SNPs. The code below shows us that for this cutoff value we have a total of 517 different clusters, each of them with a different number of isolates. For instance, we can see that there are 221 isolates that belong to "cluster number 222".

In [ ]:
# code will count how many isolates belong to a particular cluster
cutoff_2008= pd.DataFrame(pop_struc_data['cutoff_2008'].value_counts() )
cutoff_2008

If the cutoff value is high, we can expect that more isolates are grouped together with other isolates, making a lower number of clusters but most being grouped into a particular cluster. Just like before the sum of all the members for each cluster should equal to our total number of isolates (i.e, 1936).

- For example if we increase the cutoff value to say **cutoff_27236**, we can observe that most isolates (1932) would fall in "cluster 0" with only a few isolates dispersed in other clusters.


In [ ]:
# code will count how many isolates belong to a particular cluster
cutoff_27236= pd.DataFrame(pop_struc_data['cutoff_27236'].value_counts() )
cutoff_27236

Because there is no information about which cutoff value would yield us the best way to cluster our isolates or which cluster memberships are necessarily related to resistance (R) or susceptibility (S), the author included 1072 different cutoff values, each with their own set of clusters.

In [ ]:
# Observe the shape and size of the dataframe
pop_struc_data.shape

## **3) Preparing each dataset in order to be combined**

#### **a)** Matching the column name of the observations in all 3 Datasets

In [ ]:
# rename all the df with "Unnamed: 0" columns as "Isolate"
gene_presence_data = gene_presence_data.rename(columns={'Unnamed: 0' : 'Isolate'})
pop_struc_data = pop_struc_data.rename(columns={'Unnamed: 0' : 'Isolate'})

#### **b)** Changing Metadata's Year column to One Hot encoded variables
This is because even though year is a number, it is actually considered a categorical variable.

For categorical variables, it is considered a best practice to do something called "one-hot-encoding."


In [ ]:
# This prints the years the strains were isolated
year_list = metadata["Year"].unique()
year_list = year_list[np.logical_not(np.isnan(year_list))]
print(sorted(year_list))

By creating one hot encoded variables from the Year column of the metadata, each of their rows are coded the following way:
- **"0"** : Isolate was perfomed in a particular year
- **"1"** : Isolate was not performed in a particular year

In [ ]:
# creating one hot encoded variables from "Year" column to create a matrix of years
metadata_d = pd.get_dummies(metadata,columns=["Year"], dummy_na=False)
metadata_d

## **4) Making a single dataframe using all 3 sources of data**

#### **a)** Joining all data sources into a single dataframe

**Notes:**
- **lambda** is an expression used to create python functions without needing to name them specifically, in this case we are asking to merge our dataframes from left to right, so metadata_d first and then gene_presence_data.
- **reduce** is a function that allow us to repeat a particular function on a list of objects. This was done because the merge function only takes 2 dataframes at the time, but we have 3. This way we can include as many dataframes we want to merge in our **df_list**.
- The function **merge()**, allows to pass the parameter **(on ="Isolate")**, which will ensure that each isolate number is correctly matched for all 3 data sources, the parameter **(how="inner")** will make sure that isolates without a match are not included in the final dataframe.



In [ ]:
# List of all 3 data sources
df_list = [metadata_d,gene_presence_data,pop_struc_data]

# creating a single dataframe with all drugs and features available
Drug_df = reduce(lambda  left,right: pd.merge(left,right,on=['Isolate'], how='inner'), df_list)
Drug_df.head()

- Notice that the number of rows is now correctly matched with Isolate number, yielding a total of 1936 rows as in the metadata.

- Also we now have a bunch of columns that currently include:
  - **1 isolate column** these are the unique tags for each of our isolates.
  - **12 labels**, one for each drug we will try to make predictions for.
  - **18291 features** that we will be using to make prediction for the labels (isolation year, gene presence or absence and population structure).


In [ ]:
# Check out all the columns included in the final dataframe and the final shape it takes
print(Drug_df.columns) # contains all labels (drug abreviation column names)
                       # and all features (year, gene presence absence and population structure)

print("Final shape of combined dataframe",Drug_df.shape)

#### **b)** Convert the Dataframe into a CSV and save it in a folder


After running the code below, feel free to check your Drive to make sure that you have a folder named **"EColi_ML_CSV_files"** and that inside you have a csv called **"EColi_Merged_dfs.csv"**

In [ ]:
# makes a directory to save all your csv's
# Note that if you have already done this – you will get an error message saying that the File exists.
os.mkdir('/content/drive/My Drive/EColi_ML_CSV_files')

In [ ]:
# path where we will store csv data #change to any path you want
path = '/content/drive/My Drive/EColi_ML_CSV_files/'

# this code exports the dataframe into a CSV file
Drug_df.to_csv(path+"EColi_Merged_dfs.csv", index= False)

THANKS for making it this far! Now that our data is ready, we will learn how to create and train different Machine Learning algorythms using the csv file we just created. In the next Notebook, we will learn about our first algorithm for this tutorial series, [Logistic Regression](https://colab.research.google.com/drive/1iEivJY3SuMdngpakRIv5mdmUjvOi1stC?usp=sharing).